## LLM Approach: Scientific Documentation

### High-level overview
- **Goal**: Detect clusters of related bot accounts based on writing style (stylometry), optionally fusing with embedding-based similarity and labeling clusters with an LLM.
- **Pipeline**: data loading → normalization and chunking → TF‑IDF char n‑gram vectorization → randomized “impostors” AV similarity → optional embedding-based style/topical similarity → late fusion → graph thresholding + Leiden (default) community detection (fallback: Louvain) → optional topic-aware merges → optional LLM-based labeling.

### Imports and dependencies
- Core: `os`, `re`, `random`, `itertools`, `typing`.
- Scientific: `numpy`, `pandas`.
- Stylometry: `sklearn.feature_extraction.text.TfidfVectorizer` (character n‑grams), `sklearn.metrics.pairwise.cosine_similarity`.
- Graph clustering: Leiden via `leidenalg` + `igraph` (default), fallback to NetworkX `louvain_communities` + `modularity`.
- LLM stack (optional): `langchain_openai.OpenAIEmbeddings`, `langchain_openai.ChatOpenAI`, `langchain_core` prompt + JSON parser, `pydantic` for typed JSON.

Rationale: Character n‑grams capture orthography, punctuation, casing, and function words, which are stable stylistic markers; Leiden finds well-connected, dense communities and improves Louvain’s speed and quality.

### Data loading and account construction
- Loads `OPENAI_API_KEY` via `dotenv`; if missing, embedding/LLM steps are skipped.
- Reads `./PhDBotnetsDB/User_Tweet/_outputs/user_tweets_sample_enriched.csv` into `df`.
- `get_tweets_from_user(user_id)`: filters non-retweets and returns a list of `tweet_text` strings.
- Builds `accounts: Dict[str, List[str]]` mapping synthetic IDs (`acc_A` …) to tweets from selected user IDs.

Purpose: Define a small cohort of accounts with their text to run the pipeline end-to-end.

### Normalization and chunking for stylometry
- Regex normalization removes URLs and `@mentions`, lowercases, optional hashtag symbol stripping (keeps tokens), canonicalizes HTML `&amp;`, and condenses whitespace.
- `unique_norm_texts`: normalizes and deduplicates texts.
- `make_char_chunks(texts, target_chars≈8000, max_chunks=12)`: concatenates normalized tweets into several long chunks per account.
  - Longer chunks stabilize character n‑gram distributions and reduce sparsity/noise.

Scientific note: Character n‑grams are less topic-sensitive than word features and encode micro-level style (punctuation, casing, function words, emoji/hashtag usage).

### Global TF‑IDF vectorization
- `fit_vectorizer(acc_texts, ngram_range=(3,5), max_features=200k, sublinear_tf=True)`: builds a single global vocabulary across all chunks.
  - Auto-adjusts `min_df`/`max_df` to be valid given the number of documents (chunks).
  - Returns: fitted `TfidfVectorizer`, `acc_vecs` (per-account list of sparse vectors), `chunks_by_acc`, and `n_features`.

Why global vocab: Ensures comparable features and global IDF statistics across accounts.

### Randomized impostors AV similarity
- Utility functions:
  - `_rand_mask`: random feature subset mask of size `feat_frac * n_features`.
  - `_pick_chunk`: random chunk vector for an account.
  - `_cos_masked`: cosine similarity restricted to the masked features.
- `impostors_score(A_chunks, B_chunks, background, n_features, n_trials, feat_frac, bg_per_trial)`:
  - For each trial: sample features and one chunk for A and B, compute `s(A,B)`; sample `bg_per_trial` impostor accounts, compute max `s(A, impostor)`; count a win if `s(A,B)` exceeds that max.
  - Score ∈ [0,1] = wins / n_trials.
- `symmetric_impostors`: averages A→B and B→A.
- `build_similarity_matrix_AV`: full pairwise symmetric matrix for all accounts.

Scientific rationale: Randomized subspaces plus adversarial impostors yield a robust verification signal less biased by any fixed feature set or topical alignment.

Key hyperparameters: `n_trials` (stability vs compute), `feat_frac` (feature bagging strength), `bg_per_trial` (difficulty of impostor competition).

### Embedding-based style and topic representations
- Style-focused embeddings:
  - `style_prompt_prefix`: conditions the text to focus the embedder on stylistic aspects, not content topics.
  - `build_account_embeddings`: embed all chunks per account (with style prompt), mean-pool, L2-normalize to unit length.
  - `cosine_matrix_from_embeddings`: builds a [0,1]-mapped cosine matrix among accounts.
- Topic-focused embeddings:
  - `_cluster_topic_text`: concatenates normalized tweets of members of a cluster (no style prompt; we want topical semantics).
  - `build_cluster_topic_embeddings`: one embedding per cluster; vectors are L2-normalized.
- `cosine_01(a,b) = (cos(a,b) + 1)/2`: maps cosine from [-1,1] to [0,1].

Separation of concerns: style embeddings aim to capture writing habits; topic embeddings capture semantic similarity for post-hoc merges.

### Late fusion and community detection
- `fuse_scores(S_av, S_embed, alpha)`: linear fusion `S_fused = α S_av + (1−α) S_embed`.
- Community detection: threshold `S ≥ τ` to construct a weighted graph, run Leiden (default) or Louvain, compute modularity `Q`.
- `sweep_tau`: grid over τ to maximize `Q`; returns best τ, modularity, and clusters.

Scientific points: Threshold controls graph density and separability. Leiden yields well-connected, modular communities; fusion leverages complementary strengths of stylometry and embeddings.

### LLM-based cluster labeling with typed JSON
- Pydantic `ClusterLabel` schema: `cluster_name`, `style_signature` (5–10 concise markers), `cohesion_summary`, optional `notable_outliers`.
- `LABEL_PROMPT`: instructs the LLM to focus strictly on style and return STRICT JSON.
- `llm_label_clusters`: builds short style-focused samples for members, queries `ChatOpenAI`, parses via `JsonOutputParser` into `ClusterLabel` instances.

Benefit: Structured, machine-parseable labels that articulate stylistic commonalities, not topical content.

### Topic-aware cluster merging
- `merge_clusters_by_topic`:
  - Build topic embeddings per cluster.
  - Compute cluster–cluster cosine similarities; merge with union-find when similarity ≥ `topic_tau`.
  - Reindex merged groups to consecutive IDs.

Purpose: Fix over-fragmentation due to graph thresholding by merging topically similar clusters that were stylistically coherent but split across τ boundaries.

### Pairwise diagnostics
- `pair_score` and `print_pair_diagnostics`: quick inspection of select account pairs to compare `S_av`, `S_embed`, and `S_fused`.

Use case: Sanity checks and interpretability of the fusion behavior on hand-picked examples.

### Orchestration: end-to-end run
- Set seed for reproducibility.
- Validate that `accounts` is populated.
- A) Vectorize and build stylometric chunk vectors (`fit_vectorizer`).
- B) Compute AV similarity matrix `S_av` (`build_similarity_matrix_AV`).
- C) If API key available: build style embeddings per account → `S_embed`; else, proceed with AV only.
- D) Fuse (`alpha ≈ 0.6`), sweep τ for Leiden/Louvain, report best `τ` and modularity `Q` with discovered communities.
- E) If embeddings available: merge clusters by topic (`topic_tau` e.g., 0.70–0.80).
- Optionally label merged clusters via LLM and print cluster name, style signature, and summary.

### Practical notes
- Char n‑grams: robust, topic-agnostic stylometry features for short, noisy social texts.
- Impostors method: randomized subspaces and competition against many impostors calibrate a reliable [0,1] similarity.
- Fusion: late fusion is simple and interpretable; tune `alpha` with validation or stability analyses.
- Threshold τ: higher τ → tighter, smaller communities (precision↑, recall↓). Modularity sweep is a practical heuristic.
- Randomness: trials, feature masks, chunk sampling introduce variance; seeds improve reproducibility; more trials improve stability at higher cost.
- Data sufficiency: aim for ≥2–3 chunks per account for stable stylometry.
- LLM labeling: ensure samples are style-focused; validate JSON parsing and outputs.
- Complexity: AV similarity dominates runtime; consider reducing `n_trials`, `bg_per_trial`, or cohort size for large datasets.

### Possible extension
- Tune chunking (`target_chars`, `max_chunks`) for stability vs data availability.
- Adjust `(ngram_range, feat_frac, bg_per_trial, n_trials)` for compute–stability trade-offs.
- Experiment with different embedding models/providers and prompts for style/topic separation.
- Try alternative clustering (Leiden, spectral) and graph construction strategies.
- Calibrate `alpha` and `tau` using held-out linked accounts or stability criteria.



### Imports and dependencies
- Standard library, scientific stack, TF‑IDF for char n‑grams, cosine similarity.
- Graph clustering via Leiden (`leidenalg` + `igraph`); fallback to NetworkX Louvain/modularity.
- Optional LLM stack: LangChain `OpenAIEmbeddings`, `ChatOpenAI`, prompt templates, JSON parser; Pydantic for schema-validated outputs.


In [34]:
import os
import re
import random
import itertools
from typing import Dict, List, Tuple, Optional

import numpy as np
import pandas as pd
from dotenv import load_dotenv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from joblib import Parallel, delayed

import networkx as nx
from networkx.algorithms.community import louvain_communities
from networkx.algorithms.community.quality import modularity

# Optional: Leiden via igraph
try:
    import igraph as ig
    import leidenalg as la
    HAS_LEIDEN = True
except Exception:
    HAS_LEIDEN = False

from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field

### Data loading and account construction
- Load environment and API key.
- Read enriched tweets CSV.
- Helper `get_tweets_from_user` filters original tweets.
- Build `accounts` mapping from synthetic IDs to tweet lists for chosen user IDs.


In [35]:
load_dotenv()
API_KEY = os.getenv("OPENAI_API_KEY")
if not API_KEY:
    print("[WARN] OPENAI_API_KEY not set. Embeddings & LLM labeling will be disabled.")

df = pd.read_csv("./PhDBotnetsDB/User_Tweet/_outputs/user_tweets_full_enriched_english_only.csv")

def get_tweets_from_user(user_id: int) -> List[str]:
    return df.loc[(df["user_id"] == user_id) & (df["is_retweet"] == False), "tweet_text"].dropna().astype(str).tolist()

accounts: Dict[str, List[str]] = {}
for folder in df['carpeta_origen'].unique():
    accs = df[df['carpeta_origen'] == folder]['user_id'].sample(4).tolist()
    for acc in accs:
        acc_id = f"{folder} - {acc}"
        accounts[acc_id] = get_tweets_from_user(acc)
accounts.keys()

C:\Users\felip\AppData\Local\Temp\ipykernel_32728\871783049.py:6: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./PhDBotnetsDB/User_Tweet/_outputs/user_tweets_full_enriched_english_only.csv")


dict_keys(['Cresci2015-FSF - 1175035327', 'Cresci2015-FSF - 1174331750', 'Cresci2015-FSF - 1127547493', 'Cresci2015-FSF - 1175854238', 'Cresci2015-INT - 349392750', 'Cresci2015-INT - 370098498', 'Cresci2015-INT - 252647855', 'Cresci2015-TWT - 91781300', 'Cresci2015-TWT - 176096243', 'Cresci2015-TWT - 17656600', 'Cresci2015-TWT - 72186699', 'Cresci2017-SocialSpambots-1 - 467182961', 'Cresci2017-SocialSpambots-1 - 2274254095', 'Cresci2017-SocialSpambots-1 - 2267349139', 'Cresci2017-SocialSpambots-1 - 1536537319', 'Cresci2017-SocialSpambots-3 - 456934703', 'Cresci2017-SocialSpambots-3 - 531163502', 'Cresci2017-SocialSpambots-3 - 335401278', 'Cresci2017-SocialSpambots-3 - 531145445', 'Cresci2017-TraditionalSpambots-1 - 69201180', 'Cresci2017-TraditionalSpambots-1 - 104896954', 'Cresci2017-TraditionalSpambots-1 - 52996113', 'Cresci2017-TraditionalSpambots-1 - 14596967', 'Cresci2017-TraditionalSpambots-3 - 3231840636', 'Cresci2017-TraditionalSpambots-3 - 3237288540', 'Cresci2017-TraditionalS

### Normalization and chunking
- Normalize tweets: lowercase, remove URLs and `@mentions`, strip `#` (keeping tokens), convert `&amp;`, and condense whitespace.
- Deduplicate normalized texts.
- Concatenate into character-length chunks (default ≈8k chars, up to 12) to stabilize stylometric features.


In [36]:
URL_RE = re.compile(r'https?://\S+|www\.\S+')
MENTION_RE = re.compile(r'@\w+')
WS_RE = re.compile(r'\s+')

def normalize_tweet(t: str, keep_hashtags: bool = True) -> str:
    t = t.lower()
    t = URL_RE.sub(' ', t)
    t = MENTION_RE.sub(' ', t)
    if keep_hashtags:
        t = t.replace('#', '')
    t = t.replace('&amp;', '&')
    t = WS_RE.sub(' ', t).strip()
    return t

def unique_norm_texts(texts: List[str]) -> List[str]:
    seen, out = set(), []
    for s in texts:
        n = normalize_tweet(s)
        if n and n not in seen:
            seen.add(n); out.append(n)
    return out

def make_char_chunks(texts: List[str], target_chars: int = 8000, max_chunks: int = 12) -> List[str]:
    """
    Larger target_chars -> more stable AV (aim for >= 2–3 chunks per account).
    """
    msgs = unique_norm_texts(texts)
    if not msgs:
        return []
    chunks, cur, cur_len = [], [], 0
    for m in msgs:
        cur.append(m); cur_len += len(m) + 1
        if cur_len >= target_chars:
            chunks.append(" ".join(cur))
            cur, cur_len = [], 0
            if len(chunks) >= max_chunks:
                break
    if cur and len(chunks) < max_chunks:
        chunks.append(" ".join(cur))
    return chunks if chunks else [" ".join(msgs)]

### Global TF‑IDF stylometry
- Build a single character n‑gram TF‑IDF vocabulary across all chunks (`(3,5)`-grams, `sublinear_tf=True`).
- Auto-adjust `min_df`/`max_df` to be valid for the observed number of documents (chunks).
- Return the vectorizer, per-account chunk vectors, chunk texts, and feature count.


In [37]:
def fit_vectorizer(acc_texts: Dict[str, List[str]], ngram_range: Tuple[int, int] = (3, 5), max_features: int = 200_000, min_df=None, max_df=None, sublinear_tf: bool = True):
    """
    Build one global TF-IDF vocab across all account chunks (robust for AV).
    Auto-adjust min_df/max_df to avoid conflicts (max_df_docs >= min_df_docs).
    """
    docs, ids, chunks_by_acc = [], [], {}
    per_acc = {}
    for aid, texts in acc_texts.items():
        chs = make_char_chunks(texts, target_chars=8000, max_chunks=12)
        chunks_by_acc[aid] = chs[:]
        per_acc[aid] = len(chs)
        for ch in chs:
            docs.append(ch); ids.append(aid)

    n_docs = len(docs)
    if n_docs < 2:
        raise ValueError("Too few chunks. Add data or reduce target_chars.")

    umin, umax = min_df, max_df
    if min_df is None: min_df = max(1, int(0.01 * n_docs))
    if max_df is None: max_df = 0.90
    def _to_docs(val, n): return int(val * n) if isinstance(val, float) else int(val)
    max_docs = _to_docs(max_df, n_docs)
    min_docs = _to_docs(min_df, n_docs) if isinstance(min_df, int) else _to_docs(min_df, n_docs)
    if max_docs < min_docs:
        min_docs = max(1, max_docs)
        min_df = min_docs if not isinstance(umin, float) else max(umin, min_docs / n_docs)
    max_docs = _to_docs(max_df, n_docs)
    min_docs = _to_docs(min_df, n_docs) if isinstance(min_df, int) else _to_docs(min_df, n_docs)
    if max_docs < min_docs:
        max_df = min(0.99, (min_docs + 1) / n_docs) if isinstance(max_df, float) else (min_docs + 1)

    print(f"[fit_vectorizer] n_docs={n_docs} per_account={per_acc} min_df={min_df} max_df={max_df}")

    vec = TfidfVectorizer(
        analyzer="char", ngram_range=ngram_range,
        min_df=min_df, max_df=max_df, max_features=max_features,
        norm="l2", sublinear_tf=sublinear_tf
    )
    
    X = vec.fit_transform(docs)

    acc_vecs = {}
    for i, aid in enumerate(ids):
        acc_vecs.setdefault(aid, []).append(X[i])
    n_features = len(vec.get_feature_names_out())
    return vec, acc_vecs, chunks_by_acc, n_features

In [38]:
### Candidate prefilter for scalability (top-K neighbors via cosine on aggregated TF-IDF)
from scipy.sparse import vstack as sp_vstack, issparse, csr_matrix

def aggregate_account_vectors(ids: List[str], acc_vecs: Dict[str, List]) -> Tuple[np.ndarray, List[str]]:
    """
    Build a single L2-normalized mean vector per account from its chunk vectors (sparse CSR).
    """
    agg = []
    for aid in ids:
        vecs = acc_vecs[aid]
        if not vecs:
            raise ValueError(f"Account {aid} has no chunk vectors")
        if len(vecs) == 1:
            m = vecs[0]
        else:
            M = sp_vstack(vecs)
            m = M.mean(axis=0)  # returns numpy.matrix
        # ensure CSR sparse row
        if not issparse(m):
            m = csr_matrix(m)
        else:
            m = m.tocsr()
        # sklearn cosine expects L2 norm; normalize here
        denom = float(np.sqrt(m.multiply(m).sum()) + 1e-12)
        m = m / denom
        agg.append(m)
    return sp_vstack(agg), ids

def build_candidate_pairs(ids: List[str], acc_vecs: Dict[str, List], max_candidates: int = 50, min_cos: float = 0.05) -> List[Tuple[int, int]]:
    """
    Use a cheap cosine on per-account mean TF-IDF vectors to produce a sparse candidate set
    of likely similar pairs. Returns list of (i,j) with i < j.
    """
    X, _ = aggregate_account_vectors(ids, acc_vecs)
    nn = NearestNeighbors(n_neighbors=min(max_candidates + 1, len(ids)), metric="cosine", algorithm="brute")
    nn.fit(X)
    dists, indices = nn.kneighbors(X, return_distance=True)
    pairs = set()
    for i in range(len(ids)):
        for d, j in zip(dists[i], indices[i]):
            if i == j:
                continue
            cos = 1.0 - float(d)
            if cos >= min_cos:
                a, b = (i, j) if i < j else (j, i)
                pairs.add((a, b))
    return sorted(pairs)


### Impostors AV similarity and randomized subspaces
- Randomly sample feature subsets (`feat_frac`) and one chunk per account per trial.
- Compute masked cosine `s(A,B)` vs the maximum `s(A, impostor)` over sampled background accounts.
- Score is the fraction of trials where A beats impostors when paired with B; symmetrize A↔B.
- Build the full similarity matrix `S_av` across all accounts.


In [39]:
def set_random_seed(seed=42):
    random.seed(seed); np.random.seed(seed)

def _precompute_masks(n_features: int, keep_frac: float, n_masks: int, rng: np.random.Generator) -> List[np.ndarray]:
    k = max(1, int(n_features * keep_frac))
    masks = [rng.choice(n_features, size=k, replace=False) for _ in range(n_masks)]
    return masks

def _cos_masked_cols(a, b, cols: np.ndarray) -> float:
    return float(cosine_similarity(a[:, cols], b[:, cols])[0, 0])

def _pick_chunk(vecs: List, rng: np.random.Generator) -> np.ndarray:
    if not vecs: raise ValueError("Account has no chunk vectors.")
    return vecs[int(rng.integers(len(vecs)))]

def impostors_score(A_chunks: List, B_chunks: List, background: List[List],
                    n_features: int, n_trials=1200, feat_frac=0.45, bg_per_trial=70,
                    masks_cols: Optional[List[np.ndarray]] = None,
                    early_stop: bool = True, eps: float = 0.02, delta: float = 1e-3,
                    rng: Optional[np.random.Generator] = None) -> float:
    if not background: return 0.0
    rng = rng or np.random.default_rng()
    wins, poolN = 0, len(background)
    k = min(bg_per_trial, poolN)
    if masks_cols is None:
        masks_cols = _precompute_masks(n_features, feat_frac, n_trials, rng)
    bound_const = np.sqrt(0.5 * np.log(2.0 / max(delta, 1e-12))) if early_stop else None
    for t in range(1, n_trials + 1):
        cols = masks_cols[(t - 1) % len(masks_cols)]
        vA, vB = _pick_chunk(A_chunks, rng), _pick_chunk(B_chunks, rng)
        sAB = _cos_masked_cols(vA, vB, cols)
        idx = rng.choice(poolN, size=k, replace=False)
        sAI = [_cos_masked_cols(vA, _pick_chunk(background[j], rng), cols) for j in idx]
        if sAB > max(sAI, default=-1.0):
            wins += 1
        if early_stop and t >= 30:
            width = bound_const / np.sqrt(t)
            # stop when additional trials unlikely to change decision by > eps
            if width <= eps:
                break
    return wins / max(1, t)

def symmetric_impostors(A_chunks, B_chunks, background, n_features, n_trials=1200, feat_frac=0.45, bg_per_trial=70,
                        masks_cols: Optional[List[np.ndarray]] = None, early_stop: bool = True, eps: float = 0.02, delta: float = 1e-3,
                        rng: Optional[np.random.Generator] = None) -> float:
    rng = rng or np.random.default_rng()
    s1 = impostors_score(A_chunks, B_chunks, background, n_features, n_trials, feat_frac, bg_per_trial,
                         masks_cols=masks_cols, early_stop=early_stop, eps=eps, delta=delta, rng=rng)
    s2 = impostors_score(B_chunks, A_chunks, background, n_features, n_trials, feat_frac, bg_per_trial,
                         masks_cols=masks_cols, early_stop=early_stop, eps=eps, delta=delta, rng=rng)
    return 0.5 * (s1 + s2)

def build_similarity_matrix_AV(ids: List[str], acc_vecs: Dict[str, List], n_features: int, n_trials=1200, feat_frac=0.45, bg_per_trial=70,
                               seed=42, candidate_pairs: Optional[List[Tuple[int, int]]] = None, n_jobs: int = 1,
                               early_stop: bool = True, eps: float = 0.02, delta: float = 1e-3, n_masks: int = 256) -> np.ndarray:
    rng = np.random.default_rng(seed)
    n = len(ids)
    S = np.eye(n, dtype=float)
    chunks = {i: acc_vecs[i] for i in ids}
    masks_cols = _precompute_masks(n_features, feat_frac, n_masks, rng)

    pairs = candidate_pairs if candidate_pairs is not None else list(itertools.combinations(range(n), 2))

    def _compute(i, j):
        Ai, Bj = ids[i], ids[j]
        back = [chunks[k] for k in ids if k not in (Ai, Bj)]
        local_rng = np.random.default_rng(int(seed) + i * 10007 + j)
        s = symmetric_impostors(
            chunks[Ai], chunks[Bj], back, n_features,
            n_trials=n_trials, feat_frac=feat_frac, bg_per_trial=bg_per_trial,
            masks_cols=masks_cols, early_stop=early_stop, eps=eps, delta=delta, rng=local_rng
        )
        return i, j, s

    if n_jobs and n_jobs != 1:
        results = Parallel(n_jobs=n_jobs, prefer="processes")(
            delayed(_compute)(i, j) for (i, j) in pairs
        )
        for i, j, s in results:
            S[i, j] = S[j, i] = s
    else:
        for i, j in pairs:
            _, _, s = _compute(i, j)
            S[i, j] = S[j, i] = s
    return S


### Embeddings for style and topic
- Style embeddings: prefix text with a style-only instruction, embed chunks, mean-pool, L2-normalize; derive `S_embed`.
- Topic embeddings: concatenate normalized texts per cluster (no style prompt), embed and L2-normalize; used to compare and merge clusters by topic.
- Cosine values are mapped to [0,1] via `(cos + 1)/2`.


In [40]:
def style_prompt_prefix(text: str) -> str:
    return (
        "Instruction: Create an embedding that captures WRITING STYLE only "
        "(punctuation, casing, rhythm, emoji/hashtag usage, function words), not topical content.\n\n"
        "TEXT:\n" + text
    )

def build_account_embeddings(chunks_by_acc: Dict[str, List[str]], embedder) -> Dict[str, np.ndarray]:
    """Per-account L2-normalized mean of chunk embeddings (STYLE-focused)."""
    acc_vec = {}
    for aid, chunks in chunks_by_acc.items():
        if not chunks:
            acc_vec[aid] = None
            continue
        docs = [style_prompt_prefix(ch) for ch in chunks]
        embs = embedder.embed_documents(docs)      # LangChain OpenAIEmbeddings # :contentReference[oaicite:10]{index=10}
        M = np.array(embs, dtype=np.float32)
        mu = M.mean(axis=0); mu /= (np.linalg.norm(mu) + 1e-12)
        acc_vec[aid] = mu
    return acc_vec

def cosine_matrix_from_embeddings(ids: List[str], acc_emb: Dict[str, np.ndarray]) -> np.ndarray:
    mat = np.vstack([acc_emb[i] for i in ids])
    C = np.clip(mat @ mat.T, -1.0, 1.0)
    return 0.5 * (C + 1.0)

def _cluster_topic_text(cluster_members: List[str], accounts_texts: Dict[str, List[str]], max_chars_per_acc=1200) -> str:
    """
    Build a short, topic-focused sample for a whole cluster by concatenating
    (normalized) tweets of its members. We intentionally DO NOT use the style
    prompt here; we want topical semantics.
    """
    out, total = [], 0
    for aid in cluster_members:
        for t in accounts_texts[aid]:
            t = normalize_tweet(t) 
            if total + len(t) + 1 > max_chars_per_acc * len(cluster_members):
                break
            out.append(t); total += len(t) + 1
    return "\n".join(out) if out else ""

def build_cluster_topic_embeddings(clusters: Dict[int, List[str]], accounts_texts: Dict[str, List[str]], embedder) -> Dict[int, np.ndarray]:
    """
    One embedding per cluster using raw (normalized) text concatenated.
    This captures TOPIC similarity (not style).
    """
    cl_vec = {}
    docs = []
    order = []
    for cid, members in clusters.items():
        txt = _cluster_topic_text(members, accounts_texts)
        if not txt: continue
        docs.append(txt); order.append(cid)
    if not docs:
        return {}
    embs = embedder.embed_documents(docs)
    for i, cid in enumerate(order):
        v = np.array(embs[i], dtype=np.float32)
        v /= (np.linalg.norm(v) + 1e-12)
        cl_vec[cid] = v
    return cl_vec

def cosine_01(a: np.ndarray, b: np.ndarray) -> float:
    c = float(np.clip(np.dot(a, b), -1.0, 1.0))
    return 0.5 * (c + 1.0)  # map [-1,1] -> [0,1]

### Scaling notes (performance knobs)
- Top-K candidate prefilter: build cheap cosine neighbors on per-account mean TF‑IDF; only run AV on these pairs.
  - Knobs: `max_candidates` (default 50), `min_cos` (default 0.05).
- Early stopping in AV trials with Hoeffding bound.
  - Knobs: `early_stop=True`, `eps=0.02`, `delta=1e-3`, `n_trials` cap.
- Parallelism for pair scoring via joblib processes.
  - Knobs: `n_jobs` (default ≈ half cores), adjust per machine.
- Random subspace reuse: precompute column subsets to reduce per‑trial overhead.
  - Knobs: `n_masks` (default 256), `feat_frac`.
- Practical tip: increase `max_candidates` or lower `min_cos` to recover recall if you scale to many accounts.


### Fusion and community detection (Leiden/Louvain)
- Fuse stylometric AV and embedding similarities: `S_fused = α S_av + (1−α) S_embed`.
- Build a thresholded weighted graph at `S ≥ τ` and run Leiden (default) or Louvain; compute modularity `Q`.
- Sweep τ to find the partition with maximal modularity.


In [41]:
def fuse_scores(S_av: np.ndarray, S_embed: Optional[np.ndarray] = None, alpha: float = 0.7) -> np.ndarray:
    if S_embed is None:
        return S_av
    return alpha * S_av + (1.0 - alpha) * S_embed

def louvain_partition(S: np.ndarray, ids: List[str], tau: float = 0.65, resolution: float = 1.0):
    """
    Threshold S at tau -> weighted graph -> Louvain communities -> modularity Q.
    NetworkX louvain_communities & modularity are used.
    """
    G = nx.Graph(); G.add_nodes_from(ids)
    n = len(ids)
    for i in range(n):
        for j in range(i+1, n):
            w = float(S[i, j])
            if w >= tau:
                G.add_edge(ids[i], ids[j], weight=w)
    if G.number_of_edges() == 0:
        return {}, 0.0
    comms = louvain_communities(G, weight="weight", resolution=resolution, seed=42)
    Q = modularity(G, comms, weight="weight")
    clusters = {i: sorted(list(c)) for i, c in enumerate(comms)}
    return clusters, Q

def leiden_partition(S: np.ndarray, ids: List[str], tau: float = 0.65, resolution: float = 1.0):
    """
    Threshold S at tau -> weighted graph -> Leiden communities -> modularity Q.
    Implemented via igraph + leidenalg (RBConfigurationVertexPartition) with weights.
    """
    G = nx.Graph(); G.add_nodes_from(ids)
    n = len(ids)
    for i in range(n):
        for j in range(i+1, n):
            w = float(S[i, j])
            if w >= tau:
                G.add_edge(ids[i], ids[j], weight=w)
    if G.number_of_edges() == 0:
        return {}, 0.0

    # Convert to igraph
    node_to_idx = {node: i for i, node in enumerate(ids)}
    edges = []
    weights = []
    for u, v, d in G.edges(data=True):
        edges.append((node_to_idx[u], node_to_idx[v]))
        weights.append(float(d.get("weight", 1.0)))
    g = ig.Graph(n=len(ids), edges=edges, directed=False)
    g.es["weight"] = weights

    part = la.find_partition(
        g,
        la.RBConfigurationVertexPartition,
        weights="weight",
        resolution_parameter=resolution,
        n_iterations=-1,
        seed=42,
    )

    clusters = {}
    for node, comm_id in zip(ids, part.membership):
        clusters.setdefault(comm_id, []).append(node)
    clusters = {i: sorted(members) for i, members in enumerate(clusters.values())}

    Q = modularity(G, [set(m) for m in clusters.values()], weight="weight")
    return clusters, Q

def sweep_tau(S: np.ndarray, ids: List[str], taus=np.linspace(0.56, 0.72, 9), resolution=1.0, method: str = "leiden"):
    if method == "leiden":
        if not globals().get("HAS_LEIDEN", False):
            print("[WARN] Leiden not available, falling back to Louvain.")
            part_fn = louvain_partition
        else:
            part_fn = leiden_partition
    else:
        part_fn = louvain_partition

    best = {"tau": None, "Q": -1, "clusters": None}
    for tau in taus:
        cs, Q = part_fn(S, ids, tau=tau, resolution=resolution)
        if Q is not None and Q > best["Q"]:
            best = {"tau": tau, "Q": Q, "clusters": cs}
    return best

### LLM-based cluster labeling (structured JSON)
- Pydantic schema `ClusterLabel` defines the target JSON structure.
- Prompt instructs the LLM to focus on stylistic markers only.
- For each cluster: build short style-focused samples, call the LLM, and parse with `JsonOutputParser`.

Outcome: machine-parseable labels with a concise name, style signature, and cohesion summary.


In [42]:
class ClusterLabel(BaseModel):
    cluster_name: str = Field(..., description="Short English name for the botnet cluster")
    style_signature: List[str] = Field(..., description="5–10 concise style markers shared by the cluster")
    cohesion_summary: str = Field(..., description="2–3 sentences explaining WHY these accounts belong together stylistically (NOT topics)")
    notable_outliers: List[str] = Field(default_factory=list, description="Optional: account IDs that partially fit or diverge")

def _short_style_sample(texts: List[str], max_chars: int = 1500) -> str:
    buf, total = [], 0
    for t in texts:
        if not t: continue
        if total + len(t) + 1 > max_chars: break
        buf.append(t); total += len(t) + 1
    return "\n".join(buf) if buf else ""

LABEL_PROMPT = ChatPromptTemplate.from_template(
    """You are a forensic writing analyst. Label a cluster of bot accounts discovered via STYLOMETRIC similarity.
Focus ONLY on WRITING STYLE (punctuation, casing, emoji/hashtag usage, function words, character n-grams, formatting quirks), NOT topics.

Return STRICT JSON with this schema:
{schema}

For each member you receive:
- account_id
- a short style-focused sample (possibly truncated)

Members:
{members}
"""
)

def llm_label_clusters(clusters: Dict[int, List[str]],
                       accounts_texts: Dict[str, List[str]],
                       llm_model: str = "gpt-4o-mini",
                       temperature: float = 0.0) -> Dict[int, ClusterLabel]:
    if not API_KEY:
        print("[INFO] Skipping LLM labeling (no OPENAI_API_KEY).")
        return {}
    llm = ChatOpenAI(model=llm_model, temperature=temperature)
    parser = JsonOutputParser(pydantic_object=ClusterLabel)
    results: Dict[int, ClusterLabel] = {}
    for cid, members in clusters.items():
        lines = []
        for aid in members:
            sample = _short_style_sample([normalize_tweet(t) for t in accounts_texts[aid]], max_chars=1400)
            lines.append(f"- {aid}:\n\"\"\"\n{sample}\n\"\"\"")
        prompt = LABEL_PROMPT.format_messages(
            schema=parser.get_format_instructions(),
            members="\n".join(lines)
        )
        resp = llm.invoke(prompt)
        out = parser.parse(resp.content)
        if isinstance(out, dict):  
            out = ClusterLabel(**out)
        results[cid] = out
    return results

### Topic-aware cluster merging
- Build topic embeddings per cluster by concatenating normalized member texts (no style prompt).
- Merge clusters via union-find when cluster–cluster cosine ≥ `topic_tau`.
- Reindex merged groups to consecutive IDs.

Purpose: correct over-fragmentation from the thresholded graph by merging topically coherent clusters.


In [43]:
def merge_clusters_by_topic(clusters: Dict[int, List[str]], accounts_texts: Dict[str, List[str]], embedder, topic_tau: float = 0.80, verbose: bool = True) -> Dict[int, List[str]]:
    """
    Merge clusters if their TOPIC embeddings cosine ≥ topic_tau.
    - Build one "topic" embedding per cluster (concatenate normalized texts).
    - Compute cosine similarity among clusters.
    - Union-Find merge for all pairs above threshold (transitive closure).
    """
    if not clusters:
        return clusters
    # Build cluster-level topic embeddings
    cl_emb = build_cluster_topic_embeddings(clusters, accounts_texts, embedder)
    if len(cl_emb) <= 1:
        return clusters

    # Union-Find structure
    parent = {cid: cid for cid in clusters}
    def find(x):
        while parent[x] != x:
            parent[x] = parent[parent[x]]
            x = parent[x]
        return x
    def union(a, b):
        ra, rb = find(a), find(b)
        if ra != rb:
            parent[rb] = ra

    # Compare all pairs
    cids = sorted(cl_emb.keys())
    for i in range(len(cids)):
        for j in range(i+1, len(cids)):
            ci, cj = cids[i], cids[j]
            s = cosine_01(cl_emb[ci], cl_emb[cj])   
            if s >= topic_tau:
                if verbose:
                    print(f"[MERGE-TOPIC] merging clusters {ci} and {cj} (topic_sim={s:.3f} >= {topic_tau:.2f})")
                union(ci, cj)

    # Build merged mapping
    groups = {}
    for cid in clusters:
        root = find(cid)
        groups.setdefault(root, [])
        groups[root].extend(clusters[cid])

    # Reindex cluster ids 0..K-1
    merged = {}
    for new_id, (_, members) in enumerate(sorted(groups.items(), key=lambda kv: tuple(kv[1]))):
        merged[new_id] = sorted(set(members))
    return merged

### Pairwise diagnostics
Compare specific account pairs across signals:
- `S_av`: stylometric impostors similarity
- `S_embed`: style-embedding cosine (if available)
- `S_fused`: late-fused score

Use for sanity checks and qualitative inspection of fusion behavior.


In [44]:
def pair_score(ids: List[str], S: np.ndarray, a: str, b: str) -> float:
    idx = {aid: i for i, aid in enumerate(ids)}
    return float(S[idx[a], idx[b]])

def print_pair_diagnostics(ids: List[str], S_av: np.ndarray, S_embed: Optional[np.ndarray], S_fused: np.ndarray,
                           pairs: List[Tuple[str, str]]):
    for a, b in pairs:
        sav = pair_score(ids, S_av, a, b)
        se = pair_score(ids, S_embed, a, b) if S_embed is not None else None
        sf = pair_score(ids, S_fused, a, b)
        if se is None:
            print(f"[PAIR] {a} vs {b}: S_av={sav:.3f}  S_fused={sf:.3f}")
        else:
            print(f"[PAIR] {a} vs {b}: S_av={sav:.3f}  S_embed={se:.3f}  S_fused={sf:.3f}")

### Orchestration: end-to-end run
- Set random seed for reproducibility.
- Validate `accounts`.
- A) Fit TF‑IDF stylometry and chunk vectors.
- B) Compute AV similarity `S_av`.
- C) If API key: build style embeddings `S_embed`; else skip.
- D) Fuse scores and sweep τ for Louvain; report best communities.
- E) Optionally merge clusters by topic and label with an LLM.


In [45]:
if __name__ == "__main__":
    set_random_seed(42)

    if not accounts:
        raise SystemExit("Please populate the 'accounts' dict with your bots and tweets.")

    # A) Vectorize (stylometry)
    vec, acc_vecs, chunks_by_acc, n_features = fit_vectorizer(accounts, ngram_range=(3, 5), max_features=120_000)

    # B) AV similarity matrix (scaled with candidate prefilter + parallel + early stop)
    ids = list(accounts.keys())
    candidates = build_candidate_pairs(ids, acc_vecs, max_candidates=50, min_cos=0.05)
    print(f"[AV] candidate pairs: {len(candidates)} (out of {len(ids)*(len(ids)-1)//2})")
    S_av = build_similarity_matrix_AV(
        ids, acc_vecs, n_features,
        n_trials=200, feat_frac=0.40, bg_per_trial=30, seed=42,
        candidate_pairs=candidates, n_jobs=max(1, os.cpu_count() // 2),
        early_stop=True, eps=0.02, delta=1e-3, n_masks=256
    )

    S_embed = None; embedder = None
    if API_KEY:
        embedder = OpenAIEmbeddings(model="text-embedding-3-small")   
        acc_emb = build_account_embeddings(chunks_by_acc, embedder)
        S_embed = cosine_matrix_from_embeddings(ids, acc_emb)
    else:
        print("[INFO] Embeddings disabled (no OPENAI_API_KEY). Using AV signal only.")

    # D) Fusion & community detection (Leiden by default; Louvain fallback)
    alpha = 0.5
    S_fused = fuse_scores(S_av, S_embed, alpha=alpha)
    community_method = "leiden" if globals().get("HAS_LEIDEN", False) else "louvain"
    best = sweep_tau(S_fused, ids, taus=np.linspace(0.58, 0.72, 7), resolution=1.0, method=community_method)
    print(f"[{community_method.capitalize()}] best_tau={best['tau']:.2f}  modularity Q={best['Q']:.3f}")
    for cid, members in (best["clusters"] or {}).items():
        print(f"  community {cid}: {members}")

    # E) Topic-aware merging
    merged_clusters = best["clusters"] or {}
    if embedder is not None and merged_clusters:
        merged_clusters = merge_clusters_by_topic(
            merged_clusters,
            accounts_texts=accounts,
            embedder=embedder,
            topic_tau=0.8,      # if two clusters' topical cosine ≥ 0.80, merge them
            verbose=True
        )
        if merged_clusters != (best["clusters"] or {}):
            print("\n[Topic-aware merge] Final communities after merging by topic similarity:")
            for cid, members in merged_clusters.items():
                print(f"  merged community {cid}: {members}")

    labels = {}
    if API_KEY and merged_clusters:
        labels = llm_label_clusters(merged_clusters, accounts, llm_model="gpt-4o-mini", temperature=0.0)
        for cid, lab in labels.items():
            print(f"\n[Cluster {cid}] {lab.cluster_name}")
            print("  signature:", "; ".join(lab.style_signature))
            print("  summary:", lab.cohesion_summary)
            if lab.notable_outliers:
                print("  outliers:", ", ".join(lab.notable_outliers)) 

[fit_vectorizer] n_docs=219 per_account={'Cresci2015-FSF - 1175035327': 1, 'Cresci2015-FSF - 1174331750': 1, 'Cresci2015-FSF - 1127547493': 1, 'Cresci2015-FSF - 1175854238': 1, 'Cresci2015-INT - 349392750': 1, 'Cresci2015-INT - 370098498': 1, 'Cresci2015-INT - 252647855': 1, 'Cresci2015-TWT - 91781300': 1, 'Cresci2015-TWT - 176096243': 5, 'Cresci2015-TWT - 17656600': 4, 'Cresci2015-TWT - 72186699': 12, 'Cresci2017-SocialSpambots-1 - 467182961': 1, 'Cresci2017-SocialSpambots-1 - 2274254095': 1, 'Cresci2017-SocialSpambots-1 - 2267349139': 1, 'Cresci2017-SocialSpambots-1 - 1536537319': 12, 'Cresci2017-SocialSpambots-3 - 456934703': 12, 'Cresci2017-SocialSpambots-3 - 531163502': 12, 'Cresci2017-SocialSpambots-3 - 335401278': 12, 'Cresci2017-SocialSpambots-3 - 531145445': 12, 'Cresci2017-TraditionalSpambots-1 - 69201180': 9, 'Cresci2017-TraditionalSpambots-1 - 104896954': 2, 'Cresci2017-TraditionalSpambots-1 - 52996113': 12, 'Cresci2017-TraditionalSpambots-1 - 14596967': 12, 'Cresci2017-Tra